# Tutorial 00: P1 Finite Elements for the Poisson Equation

In this exercise, we will solve our first PDE:

$$
\begin{aligned}
-\Delta u & = f && in\ \Omega \\
u & = g && on\ \partial\Omega
\end{aligned}
$$

Again, we include all the Dune sources through one convenience header:

In [ ]:
#include<dune/jupyter.hh>

The simulation code written below will depend on a number of runtime parameters that are provided in a configuration file `tutorial00.ini`. You find that file in the `notebooks` directory. Dune uses a nested data structured called `ParameterTree` for these configurations. In the following we parse the configuration, output it to the notebook and alter it from within the notebook:

In [ ]:
Dune::ParameterTree ptree;
Dune::ParameterTreeParser::readINITree("tutorial00.ini", ptree);

In [ ]:
ptree

In [ ]:
ptree["grid.refinement"] = "0";

We are using a two-dimensional, unstructured mesh for this simulation. The mesh resolves the unit square $\Omega = [0,1]^2$. The file is specified in the `unitsquare.msh` file that was generated using Gmsh.

In [ ]:
const int dim = 2;
using Grid = Dune::UGGrid<dim>;
std::string filename = ptree.get("grid.twod.filename", "unitsquare.msh");
Dune::GridFactory<Grid> factory;
Dune::GmshReader<Grid>::read(factory,filename,true,true);
std::unique_ptr<Grid> grid(factory.createGrid());
grid->globalRefine(ptree.get<int>("grid.refinement"));
auto gv = grid->leafGridView();
using GV = decltype(gv);

In [ ]:
grid

With Dune making heavy use of C++ templates, the underlying floating point type of the simulation can be chosen quite freely. Looking for a solution $u: \Omega\rightarrow\mathbb{R}$, the *domain field* type is the floating point type used to realize $\Omega$ where as the *range field* type is used to realize $\mathbb{R}$. `DF` is defined by the grid implementation (usually to `double`), but we can freely choose `RF` - typically also to `double`:

In [ ]:
using DF = Grid::ctype;
using RF = double;

In [ ]:
using FEM = Dune::PDELab::PkLocalFiniteElementMap<GV,DF,RF,1>;
FEM fem(gv);
using CON = Dune::PDELab::ConformingDirichletConstraints;
using VBE = Dune::PDELab::ISTL::VectorBackend<>;
using GFS = Dune::PDELab::GridFunctionSpace<GV,FEM,CON,VBE>;
GFS gfs(gv,fem);
gfs.name("P1");

In [ ]:
using Z = Dune::PDELab::Backend::Vector<GFS,RF>;
Z z(gfs);

In [ ]:
auto g = Dune::PDELab::makeGridFunctionFromCallable(
    gv,
    [](const auto& x){
        RF s=0.0;
        for (std::size_t i=0; i<x.size(); i++)
            s+=x[i]*x[i];
        return s;
    }
);
Dune::PDELab::interpolate(g,gfs,z);

In [ ]:
using CC = typename GFS::template ConstraintsContainer<RF>::Type;
CC cc;

In [ ]:
auto b = Dune::PDELab::makeBoundaryConditionFromCallable(
    gv,
    [](const auto& x){ return true; }
);
Dune::PDELab::constraints(b, gfs, cc);
std::cout << "constrained dofs=" << cc.size() << " of "
          << gfs.globalSize() << std::endl;

In [ ]:
auto f = Dune::PDELab::makeGridFunctionFromCallable(
    gv,
    [](const auto& x){
        return Dune::FieldVector<RF,1>(-2.0*x.size());
    }
);;

Next, we will construct the local operator. We include the local operator implementation for P1-conforming finite elements from the file `poissonp1.hh`. You find this file in the `notebooks` folder. After making changes to the file, you need to restart and rerun this notebook.

In [ ]:
#include"poissonp1.hh"

using LOP = PoissonP1<decltype(f),FEM>;
LOP lop(f, fem.find(*gv.template begin<0>()));

In [ ]:
using MBE = Dune::PDELab::ISTL::BCRSMatrixBackend<>;
MBE mbe(1<<(dim+1)); // guess nonzeros per row
using GO = Dune::PDELab::GridOperator<
  GFS,GFS,  /* ansatz and test space */
  LOP,      /* local operator */
  MBE,      /* matrix backend */
  RF,RF,RF, /* domain, range, jacobian field type*/
  CC,CC     /* constraints for ansatz and test space */
  >;
GO go(gfs,cc,gfs,cc,lop,mbe);

In [ ]:
using LS = Dune::PDELab::ISTLBackend_SEQ_CG_AMG_SSOR<GO>;
LS ls(100,2);

In [ ]:
using SLP = Dune::PDELab::StationaryLinearProblemSolver<GO,LS,Z>;
SLP slp(go,ls,z,1e-10);
slp.apply();

In [ ]:
using ZDGF = Dune::PDELab::DiscreteGridFunction<GFS,Z>;
ZDGF zdgf(gfs,z);
Z w(gfs); // Lagrange interpolation of exact solution
Dune::PDELab::interpolate(g,gfs,w);
ZDGF wdgf(gfs,w);
Dune::VTKWriter<GV> vtkwriter(gv,Dune::VTK::conforming);
using VTKF = Dune::PDELab::VTKGridFunctionAdapter<ZDGF>;
vtkwriter.addVertexData(std::shared_ptr<VTKF>(new VTKF(zdgf,"fesol")));
vtkwriter.addVertexData(std::shared_ptr<VTKF>(new VTKF(wdgf,"exact")));
vtkwriter.write(ptree.get("output.filename","output"), Dune::VTK::appendedraw);